In [1]:
%pip install graphrag_sdk==0.1.3b0
%pip install -e /workspaces/autogen

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///workspaces/autogen
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for autogen (pyproject.toml) ... done
  Created wheel for autogen: filename=autogen-0.3.0-0.editable-py3-none-any.whl size=16273 sha256=e98cc4d562793824fed6eb82a0882d331170bf9161260ff45ff3dc32570f0bb5
  Stored in directory: /tmp/pip-ephem-wheel-cache-0wb4pw7w/wheels/d4/86/1f/a538740a449c67aa3ff8343698c29d70fc94236f70bde92144
Successfully built autogen
  Attempting uninstall: autogen
    Found existing installation: autogen 0.3.0
    Uninstalling autogen-0.3.0:
      Successfully uninstalled autogen-0.3.0
Not

In [2]:
import os

import autogen

config_list = autogen.config_list_from_json(env_or_file="../OAI_CONFIG_LIST")
os.environ["OPENAI_API_KEY"] = config_list[0]["api_key"]

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from graphrag_sdk.schema import Schema

from autogen.agentchat.contrib.graph_rag.document import Document, DocumentType
from autogen.agentchat.contrib.graph_rag.falkor_graph_query_engine import FalkorGraphQueryEngine

# Auto generate graph schema from unstructured data
input_path = "../test/agentchat/contrib/graph_rag/the_matrix.txt"

movie_schema = Schema()
actor = movie_schema.add_entity("Actor").add_attribute("name", str, unique=True)
movie = movie_schema.add_entity("Movie").add_attribute("title", str, unique=True)
movie_schema.add_relation("ACTED", actor, movie)

query_engine = FalkorGraphQueryEngine(
    name="IMDB",
    host="172.18.0.3",
    port=6379,
    schema=movie_schema,
)

input_documents = [Document(doctype=DocumentType.TEXT, path_or_url=input_path)]

query_engine.init_db(input_doc=input_documents)

# Query your data
question = "Name a few actors who've played in 'The Matrix'"
answer = query_engine.query(question)
print(f"Answer: {answer}")